# .tflite experiments

## Building flatc

- Visit the [Flatbuffer repository](https://github.com/google/flatbuffers)
- Tutorials are [here](https://google.github.io/flatbuffers/flatbuffers_guide_building.html)
```
git clone https://github.com/google/flatbuffers
cmake -G "Unix Makefiles" -DCMAKE_BUILD_TYPE=Release
make
```

## Generate TF Lite schema
```
wget https://raw.githubusercontent.com/tensorflow/tensorflow/master/tensorflow/lite/schema/schema.fbs
pip install flatbuffers
./flatbuffers/flatc -p schema.fbs
```
The generated python code is under `tflite/`

## Generate the .tflite File

We are experimenting with a network with its architecture shown below: ![](https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/image/mlp_mnist.png)

The code to construct, train and export the model as a `.tflite` can be viewed [here](https://gist.github.com/neil-tan/6d77ee4029dff09d33830d716ac26a3a).
The pre-generated TF Lite file is included in the current repository as `converted_model.tflite`.




## Python Experiments
Referencing [tutorial1](https://google.github.io/flatbuffers/flatbuffers_guide_tutorial.html) and [tutorial2](https://google.github.io/flatbuffers/flatbuffers_guide_use_python.html)

Flatbuffer Import

In [ ]:
import flatbuffers
import tflite.Model as tf_model

In [ ]:
buf = open('converted_model.tflite', 'rb').read()
buf = bytearray(buf)
model = tf_model.Model.GetRootAsModel(buf, 0)
print(tf_graph.Version())

In [ ]:
# getting subgraph(s)
subgraphs_len = model.SubgraphsLength()
print(subgraphs_len)

In [ ]:
subgraph = model.Subgraphs(0)
print(subgraph)

Operators

In [ ]:
#op code lookup object
op_codes = []
for i in range(0, model.OperatorCodesLength()):
    op_code =  model.OperatorCodes(i)
    op_codes.append(op_code)
    
print(op_codes)

In [ ]:
# printing the built-in op codes
# see BuiltinOperator.py

import tflite.BuiltinOperator as BuiltinOperator
builtin_ops = {v: k for k, v in BuiltinOperator.BuiltinOperator.__dict__.items()}    

print(subgraph.OperatorsLength(), "operators:")

for i in range(0, subgraph.OperatorsLength()):
    op = subgraph.Operators(i)
    opIndex = op.OpcodeIndex()
    op_code = op_codes[opIndex]
    builtin_code = op_code.BuiltinCode()
    print(builtin_ops[builtin_code])
